In [1]:
import pandas as pd
import numpy as np
import os
import re
import requests
import time
import bs4
#import lxml


In [13]:
url = "https://fortnitetracker.com/events/epicgames_S21_PgodCup_NAW?window=S21_PgodCup_NAW_Round1"
data = requests.get(url).text
soup = bs4.BeautifulSoup(data, 'html.parser')
soup


<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no, shrink-to-fit=no" name="viewport"/>
<meta content="#080808" name="msapplication-navbutton-color"/>
<meta content="#080808" name="theme-color"/>
<meta content="spvb9FJmktijuUuNiAoicMXyQg8X8cuqtQSz_A-LNI8" name="google-site-verification">
<meta content="a8a63fb085946997e77449771e3d271cac8a652c" name="naver-site-verification">
<meta content="app-id=network.tracker.app" name="google-play-app"/>
<meta content="app-id=1287696482" name="apple-itunes-app"/>
<link href="https://fortnitetracker.com" rel="preconnect"/>
<link href="https://cdn.thetrackernetwork.com" rel="preconnect"/>
<link href="https://fonts.googleapis.com" rel="preconnect"/>
<link href="https://cdnjs.cloudflare.com" rel="preconnect"/>
<link href="https://files.fortnitetracker.com" rel="preconnect"/>
<link href="https://code.ionicframework.com" rel="preconnect"/>
<link hr

In [54]:
#Players on the Leaderboard on Esport Orgs
players = soup.find_all('script')
players = players[55].text
pattern = r'"name": (.+),'
re.findall(pattern, players)

['"Zno"',
 '"Nach"',
 '"Hov"',
 '"evol"',
 '"remi"',
 '"Kruyor"',
 '"Cooper"',
 '"Bacon"',
 '"Bacca"',
 '"On1"',
 '"Elder"',
 '"Kickuh"',
 '"defiable"',
 '"Cyperzz"',
 '"Mudzi"',
 '"OliverOG"',
 '"Knzo"',
 '"THORIK"',
 '"Ritual"',
 '"Dynamic"',
 '"Caleb"',
 '"Reet"',
 '"favs"',
 '"Epikwhale"',
 '"Rainez"',
 '"Wukong"',
 '"Gabe"',
 '"Kenshi"',
 '"Kaboo"',
 '"Statix"',
 '"Jagveer"',
 '"chicken"',
 '"Trey509"',
 '"Cwintrr"',
 '"Nigel"',
 '"Jay"',
 '"Zagou"',
 '"goma"',
 '"4DRStorm"',
 '"Snacky"',
 '"Middi"',
 '"Revised"',
 '"danny bad bloom"',
 '"jman"',
 '"CZR"',
 '"Criizux"',
 '"kimchi"',
 '"Ming"',
 '"Squishy"',
 '"Pollo"',
 '"ChickenItza"',
 '"andyvencha"',
 '"Jadenx "',
 '"duckie"',
 '"Echo"',
 '"Clukz"',
 '"Cold"',
 '"Staticcs"',
 '"Lancers"',
 '"Veer"',
 '"Neeqo"',
 '"verT"',
 '"Tann"',
 '"Victim"',
 '"Nyndee"']

In [15]:
stats = soup.find_all('script')[51].text.split('"entries": [')[1]
stats

'\r\n    {\r\n      "eventId": "epicgames_S21_PgodCup_NAW",\r\n      "eventWindowId": "S21_PgodCup_NAW_Round1",\r\n      "gameId": "Fortnite",\r\n      "percentile": 0.0,\r\n      "pointBreakdown": {\r\n        "PLACEMENT_STAT_INDEX:14": {\r\n          "timesAchieved": 7.0,\r\n          "pointsEarned": 7.0\r\n        },\r\n        "PLACEMENT_STAT_INDEX:4": {\r\n          "timesAchieved": 6.0,\r\n          "pointsEarned": 12.0\r\n        },\r\n        "PLACEMENT_STAT_INDEX:7": {\r\n          "timesAchieved": 6.0,\r\n          "pointsEarned": 6.0\r\n        },\r\n        "PLACEMENT_STAT_INDEX:3": {\r\n          "timesAchieved": 6.0,\r\n          "pointsEarned": 12.0\r\n        },\r\n        "PLACEMENT_STAT_INDEX:10": {\r\n          "timesAchieved": 6.0,\r\n          "pointsEarned": 6.0\r\n        },\r\n        "PLACEMENT_STAT_INDEX:6": {\r\n          "timesAchieved": 6.0,\r\n          "pointsEarned": 6.0\r\n        },\r\n        "PLACEMENT_STAT_INDEX:11": {\r\n          "timesAchieved": 

In [55]:
#regex to find "pointsEarned": 107,\r\n      "rank": 100,\r\n  poionts and rank
pattern = r'"sessionStats": [\d\w\s\{\}.:",]+'

teamStats = re.findall(pattern, stats)
cleanStats = []
for team in teamStats:
    pattern = r'"elims": ([0-9]+),'
    elims = re.findall(pattern, team)[0]
    pattern = r'"wins": ([0-9]+),'
    wins = re.findall(pattern, team)[0]
    pattern = r'"matches": ([0-9]+),'
    matches = re.findall(pattern, team)[0]
    pattern = r'"avgPoints": ([0-9.]+),'
    avgPoints = re.findall(pattern, team)[0]
    pattern = r'"avgPlace": ([0-9.]+),'
    avgPlace = re.findall(pattern, team)[0]
    pattern = r'"avgElims": ([0-9.]+),'
    avgElims = re.findall(pattern, team)[0]
    pattern = r'"kdRatio": ([0-9.]+),'
    kdRatio = re.findall(pattern, team)[0]
    cleanStats.append({'elims': elims, 'wins': wins, 'matches': matches, 'avgPoints': avgPoints, 'avgElims': avgElims, 'avgPlace': avgPlace, 'kdRatio': kdRatio})

pattern = r'"pointsEarned": ([0-9]+),'
points = re.findall(pattern, stats)
pattern = r'"rank": ([0-9]+),'
rank = re.findall(pattern, stats)

df = pd.DataFrame(cleanStats)
df['points'] = points
df['rank'] = rank
df = df.astype(float)

#Finding Elimination/Placement Point Distribution Stats
elimLine = soup.find_all('div', {'class': 'fne-scores__entry'})[-1].text
pattern = r'\nEach Elimination\n\+ ([0-9]+)\n'
elimVal = float(re.findall(pattern, elimLine)[0])
df['elimPoints'] = df['elims'] * elimVal
df['placementPoints'] = df['points'] - df['elimPoints']
df['elimPtsProportion'] = df['elimPoints']/df['points']
df['placementPtsProportion'] = df['placementPoints']/df['points']
df['winrate'] = df['wins']/df['matches']
df

,elims,wins,matches,avgPoints,avgElims,avgPlace,kdRatio,points,rank,elimPoints,placementPoints,elimPtsProportion,placementPtsProportion,winrate
0,122.0,4.0,9.0,29.56,13.56,6.89,24.40,266.0,1.0,122.0,144.0,0.458647,0.541353,0.444444
1,104.0,4.0,7.0,35.71,14.86,2.57,34.67,250.0,2.0,104.0,146.0,0.416000,0.584000,0.571429
2,88.0,3.0,9.0,24.89,9.78,6.44,14.67,224.0,3.0,88.0,136.0,0.392857,0.607143,0.333333
3,124.0,1.0,10.0,22.30,12.40,12.70,13.78,223.0,4.0,124.0,99.0,0.556054,0.443946,0.100000
4,76.0,5.0,9.0,24.67,8.44,7.67,19.00,222.0,5.0,76.0,146.0,0.342342,0.657658,0.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,69.0,1.0,9.0,17.67,7.67,10.44,8.62,159.0,96.0,69.0,90.0,0.433962,0.566038,0.111111
96,82.0,1.0,10.0,15.80,8.20,13.10,9.11,158.0,97.0,82.0,76.0,0.518987,0.481013,0.100000
97,90.0,0.0,10.0,15.80,9.00,13.30,9.00,158.0,98.0,90.0,68.0,0.569620,0.430380,0.000000
98,79.0,0.0,10.0,15.80,7.90,11.20,7.90,158.0,99.0,79.0,79.0,0.500000,0.500000,0.000000


In [56]:
df.sort_values(by='winrate', ascending = False)

,elims,wins,matches,avgPoints,avgElims,avgPlace,kdRatio,points,rank,elimPoints,placementPoints,elimPtsProportion,placementPtsProportion,winrate
81,86.0,3.0,4.0,40.25,21.50,7.50,86.00,161.0,82.0,86.0,75.0,0.534161,0.465839,0.750000
1,104.0,4.0,7.0,35.71,14.86,2.57,34.67,250.0,2.0,104.0,146.0,0.416000,0.584000,0.571429
4,76.0,5.0,9.0,24.67,8.44,7.67,19.00,222.0,5.0,76.0,146.0,0.342342,0.657658,0.555556
62,44.0,4.0,8.0,20.75,5.50,7.00,11.00,166.0,63.0,44.0,122.0,0.265060,0.734940,0.500000
0,122.0,4.0,9.0,29.56,13.56,6.89,24.40,266.0,1.0,122.0,144.0,0.458647,0.541353,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,60.0,0.0,10.0,16.10,6.00,10.10,6.00,161.0,90.0,60.0,101.0,0.372671,0.627329,0.000000
61,88.0,0.0,10.0,16.70,8.80,13.00,8.80,167.0,62.0,88.0,79.0,0.526946,0.473054,0.000000
97,90.0,0.0,10.0,15.80,9.00,13.30,9.00,158.0,98.0,90.0,68.0,0.569620,0.430380,0.000000
98,79.0,0.0,10.0,15.80,7.90,11.20,7.90,158.0,99.0,79.0,79.0,0.500000,0.500000,0.000000


In [57]:
def get_content(url):
    """
    get_book that takes in the url of a 'Plain Text UTF-8' book and 
    returns a string containing the contents of the book.

    The function should satisfy the following conditions:
        - The contents of the book consist of everything between 
        Project Gutenberg's START and END comments.
        - The contents will include title/author/table of contents.
        - You should also transform any Windows new-lines (\r\n) with 
        standard new-lines (\n).
        - If the function is called twice in succession, it should not 
        violate the robots.txt policy.

    :Example: (note '\n' don't need to be escaped in notebooks!)
    >>> url = 'http://www.gutenberg.org/files/57988/57988-0.txt'
    >>> book_string = get_book(url)
    >>> book_string[:20] == '\\n\\n\\n\\n\\nProduced by Chu'
    True
    """
    time.sleep(5)
    pg = requests.get(url)
    if pg.status_code == 200:
        page = str(pg.content)
        return page
